In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler,StandardScaler,OneHotEncoder
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

from sklearn.ensemble import GradientBoostingClassifier

In [2]:
# load the df
df = pd.read_csv('../../data/df_dummies.csv')

In [3]:
# train_test_split
features = df.drop(columns=['heart_attack_risk','income'])
target = df['heart_attack_risk']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=0)

In [4]:
# normalization
normalizer = MinMaxScaler()
normalizer.fit(X_train)
X_train_norm = normalizer.transform(X_train)
X_test_norm = normalizer.transform(X_test)

X_train_norm = pd.DataFrame(X_train_norm,columns=X_train.columns)
X_test_norm = pd.DataFrame(X_test_norm,columns=X_test.columns)

In [5]:
# smote
from imblearn.over_sampling import SMOTE

In [6]:
sm = SMOTE(random_state=123,sampling_strategy=1.0)
X_train_SMOTE, y_train_SMOTE = sm.fit_resample(X_train_norm,y_train)

In [7]:
y_train_SMOTE.value_counts()

heart_attack_risk
0    4510
1    4510
Name: count, dtype: int64

# Gradient Boosting

In [8]:
gb_reg = GradientBoostingClassifier(max_depth=20,
                                    n_estimators=100,
                                    random_state=10)

gb_reg.fit(X_train_SMOTE, y_train_SMOTE)

GradientBoostingClassifier(max_depth=20, random_state=10)

In [9]:
pred_gb = gb_reg.predict(X_test_norm)

In [10]:
print(classification_report(y_test, pred_gb))

              precision    recall  f1-score   support

           0       0.64      0.79      0.70      1114
           1       0.37      0.21      0.27       639

    accuracy                           0.58      1753
   macro avg       0.50      0.50      0.49      1753
weighted avg       0.54      0.58      0.54      1753



In [11]:
confusion_matrix(y_test,pred_gb)

array([[881, 233],
       [505, 134]], dtype=int64)

In [12]:
import plotly.express as px

In [13]:
cm = pd.DataFrame(confusion_matrix(y_test, pred_gb))
# Rename columns to predicted values - 0 = No Risk, 1 = Risk
cm.rename({0: 'No - True', 1: 'Yes - True'}, axis=1, inplace=True)
# Rename rows to real values - 0 = No Risk, 1 = Risk
cm.rename({0: 'No - Pred', 1: 'Yes - Pred'}, axis=0, inplace=True)
px.imshow(cm, text_auto=True, color_continuous_scale='RdBu', color_continuous_midpoint=0)